In [1]:
# 필요 라이브러리 import
import os
from typing import Tuple, Any, Callable, List, Optional, Union
import pandas as pd
import cv2
import torch
import timm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
#from albumentations import Compose, Resize, Normalize
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm
import torch.nn.functional as F
from timm.loss import SoftTargetCrossEntropy

# FocalLoss 정의
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return torch.mean(F_loss)

# Custom Dataset 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, root_dir: str, info_df: pd.DataFrame, transform: Callable, is_inference: bool = False):
        self.root_dir = root_dir
        self.transform = transform
        self.is_inference = is_inference
        self.image_paths = info_df['image_path'].tolist()
        if not self.is_inference:
            self.targets = info_df['target'].tolist()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index) -> Union[Tuple[torch.Tensor, int], torch.Tensor]:
        img_path = os.path.join(self.root_dir, self.image_paths[index])
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        # Convert to grayscale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)  # keep 3 channels for pre-trained models
        
        # Albumentations expects a dictionary {'image': image}
        transformed = self.transform(image=image)
        image = transformed['image']  # Extract the transformed image tensor

        # image = self.transform(image=image)
        
        if self.is_inference:
            return image
        else:
            target = self.targets[index]
            return image, target

## brightness_limit=(-0.2, -0.2) : 밝기를 -0.2로 고정합니다. 이 값은 이미지 밝기를 20% 어둡게 만들겠다는 의미
## contrast_limit=0 : 대비는 조정하지 않겠다는 의미
## p=0.9 : 90% 확률로 이 변환을 적용하겠다는 뜻
## 만약 10% 어둡게 만들고 싶다면 brightness_limit=(-0.1, -0.1)로 변경


# Albumentations 변환기
def get_train_transforms():
    return A.Compose([
        A.Rotate(limit=15), # 최대 15도 회전
        A.RandomBrightnessContrast(brightness_limit=(-0.2, -0.2), contrast_limit=0, p=0.9), # 20% 어둡게 # 대환이 형이 찾아낸 방법 # 좀 더 이미지가 명확하게 보임..
        A.GaussianBlur(blur_limit=(3, 5), p=0.3), # 약간의 블러 추가
        A.OneOf([A.Emboss(p=0.3), A.Sharpen(p=0.3)], p=0.3),  # Emboss & Sharpen
        A.ElasticTransform(alpha=0.3, sigma=10, alpha_affine=5, p=0.3), # Elastic Transform (강도 조정)  
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
        ToTensorV2() # ToTensorV2가 마지막으로 와야 함!!
    ])
    
# Albumentations 변환기
def get_val_transforms():
    return A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2() # ToTensorV2가 마지막으로 와야 함!!
    ])


# 모델 앙상블 클래스 정의
class EnsembleModel(nn.Module):
    def __init__(self, models: List[nn.Module]):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)

    def forward(self, x):
        outputs = [model(x) for model in self.models]
        output = torch.stack(outputs).mean(0)
        return output

# k-fold 교차 검증 설정
def get_dataloaders(train_info, traindata_dir, kfold=5):
    skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=42)
    fold_dataloaders = []
    for train_idx, val_idx in skf.split(train_info, train_info['target']):
        train_df = train_info.iloc[train_idx]
        val_df = train_info.iloc[val_idx]
        train_dataset = CustomDataset(traindata_dir, train_df, get_train_transforms())
        val_dataset = CustomDataset(traindata_dir, val_df, get_val_transforms())
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
        fold_dataloaders.append((train_loader, val_loader))
    return fold_dataloaders

# 학습 함수 정의
def train_model(model, train_loader, val_loader, device, optimizer, scheduler, loss_fn, epochs):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            images, targets = images.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        val_loss = validate_model(model, val_loader, device, loss_fn)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        scheduler.step()

# 검증 함수 정의
def validate_model(model, val_loader, device, loss_fn):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in tqdm(val_loader, desc="[Validation]"):
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, targets)
            val_loss += loss.item()
    return val_loss / len(val_loader)

# 최종 추론
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for images in tqdm(test_loader, desc="[Inference]"):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)
            predictions.extend(preds.cpu().numpy())
    return predictions

# 결과 저장 함수
#def save_predictions(predictions, output_path="resnet101_efficientnet_b3_seresnet50_etc.csv"):
#    submission_df = pd.DataFrame({'ImageId': list(range(len(predictions))), 'Label': predictions})
#    submission_df.to_csv(output_path, index=False)

/opt/conda/envs/blueberry/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 주요 실행 부분
if __name__ == "__main__":
    # 학습 데이터 로드
    traindata_dir = './data/train'
    traindata_info_file = './data/train.csv'
    train_info = pd.read_csv(traindata_info_file)
    num_classes = len(train_info['target'].unique())

    # 모델 앙상블 설정
    model1 = timm.create_model('resnet101', pretrained=True, num_classes=num_classes)
    model2 = timm.create_model('efficientnet_b3', pretrained=True, num_classes=num_classes)
    model3 = timm.create_model('seresnet50', pretrained=True, num_classes=num_classes)
    ensemble_model = EnsembleModel([model1, model2, model3])

    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # k-fold 데이터 로더 생성
    fold_dataloaders = get_dataloaders(train_info, traindata_dir)

    # 옵티마이저, 스케줄러, 손실 함수 설정
    optimizer = optim.Adam(ensemble_model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(fold_dataloaders[0][0]), epochs=30) # warm up scheduler 구현
    loss_fn = FocalLoss()

    # 학습 진행 (k-fold)
    epochs = 30
    for fold, (train_loader, val_loader) in enumerate(fold_dataloaders):
        print(f"Fold {fold+1}/{len(fold_dataloaders)}")
        train_model(ensemble_model, train_loader, val_loader, device, optimizer, scheduler, loss_fn, epochs)

    # 추론 및 결과 저장
    testdata_dir = './data/test'
    testdata_info_file = './data/test.csv'
    test_info = pd.read_csv(testdata_info_file)
    test_dataset = CustomDataset(testdata_dir, test_info, get_val_transforms(), is_inference=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    predictions = inference(ensemble_model, test_loader, device)
    #save_predictions(predictions)

Fold 1/5


[Validation]: 100%|██████████| 47/47 [00:25<00:00,  1.83it/s]


Epoch 1/30, Train Loss: 1151.9333, Val Loss: 6.0197


[Validation]: 100%|██████████| 47/47 [00:24<00:00,  1.89it/s]


Epoch 2/30, Train Loss: 1081.9744, Val Loss: 5.4268


...

[Validation]: 100%|██████████| 47/47 [00:25<00:00,  1.82it/s]


Epoch 29/30, Train Loss: 0.8676, Val Loss: 0.0215


[Validation]: 100%|██████████| 47/47 [00:25<00:00,  1.82it/s]


Epoch 30/30, Train Loss: 1.1190, Val Loss: 0.0207


[Inference]: 100%|██████████| 157/157 [01:26<00:00,  1.82it/s]


In [3]:
# 모든 클래스에 대한 예측 결과를 하나의 문자열로 합침
test_info['target'] = predictions
test_info = test_info.reset_index().rename(columns={"index": "ID"})
test_info

,ID,image_path,target
0,0,0.JPEG,328
1,1,1.JPEG,414
2,2,2.JPEG,493
3,3,3.JPEG,17
4,4,4.JPEG,388
...,...,...,...
10009,10009,10009.JPEG,235
10010,10010,10010.JPEG,86
10011,10011,10011.JPEG,307
10012,10012,10012.JPEG,400


In [4]:
# DataFrame 저장
test_info.to_csv("resnet101_efficientnet_b3_seresnet50_etc.csv", index=False)